## Using HOG Features and a SVM

### Training

In [7]:
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from skimage import io, color
import tifffile
import numpy as np
import os
import glob


def load_rgb_images_and_labels(base_dir):
    labels = []
    features = []
    label_names = os.listdir(base_dir)
    
    for label_name in label_names:
        dir_path = os.path.join(base_dir, label_name)
        image_files = glob.glob(os.path.join(dir_path, "*.jpg"))
        
        for image_file in image_files:
            image = io.imread(image_file)
            
            # convert the image to grayscale as HOG needs a single channel image
            if image.ndim == 3:
                image = color.rgb2gray(image)
            
            # extract HOG features
            fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1,1), block_norm='L2-Hys')
            
            features.append(fd)
            labels.append(label_name) # using dir name as label
    return np.array(features), np.array(labels), label_names

def load_ms_images_and_labels(base_dir):
    labels = []
    features = []
    label_names = [d for d in os.listdir(base_dir) if not d.startswith('.')] ## ignoring macs .DS_Store file

    
    for label_name in label_names:
        dir_path = os.path.join(base_dir, label_name)
        image_files = glob.glob(os.path.join(dir_path, "*.tif"))
        
        for image_file in image_files:
            # TIFF files can contain multiple bands
            image = tifffile.imread(image_file)
        
            
            # averaging the bands
            if image.ndim == 3:
                image = np.mean(image, axis=2)
            
            # Extract HOG features
            fd = hog(image, orientations=8, pixels_per_cell=(16,16), cells_per_block=(1,1), block_norm='L2-Hys')
            
            features.append(fd)
            labels.append(label_name)
    return np.array(features), np.array(labels), label_names

base_dir_rgb = "./data/train/EuroSAT_RGB"
X_rgb, y_rgb, label_names_rgb = load_rgb_images_and_labels(base_dir_rgb)

base_dir_ms = "./data/train/EuroSAT_MS"
X_ms, y_ms, label_names_ms = load_ms_images_and_labels(base_dir_ms)

# Ensure the labels match between RGB and MS datasets
assert set(label_names_rgb) == set(label_names_ms)

# Combine the datasets
X = np.concatenate((X_rgb, X_ms), axis=0)
y = np.concatenate((y_rgb, y_ms), axis=0)

# Encode labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the classifier
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

# Predict and evaluate the classifier
y_pred = clf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))  # Use le.classes_ to get the original label names


Accuracy:  0.528425925925926
                      precision    recall  f1-score   support

          AnnualCrop       0.64      0.76      0.70      1192
              Forest       0.49      0.60      0.54      1197
HerbaceousVegetation       0.36      0.35      0.35      1195
             Highway       0.65      0.59      0.62      1059
          Industrial       0.55      0.42      0.48       962
             Pasture       0.28      0.13      0.18       779
       PermanentCrop       0.41      0.31      0.35       979
         Residential       0.44      0.74      0.55      1206
               River       0.75      0.69      0.72      1045
             SeaLake       0.64      0.52      0.57      1186

            accuracy                           0.53     10800
           macro avg       0.52      0.51      0.51     10800
        weighted avg       0.53      0.53      0.52     10800


### Testing

In [12]:
import pandas as pd

test_npy_path = "./data/testset/testset/"
test_csv_path = "./data/test.csv"
test_df = pd.read_csv(test_csv_path)

def preprocess_and_extract_features(image_array):
    # Check if the image is RGB or multispectral based on the number of channels
    if image_array.ndim == 3:
        if image_array.shape[2] == 3:  # RGB image
            image_gray = color.rgb2gray(image_array)
        elif image_array.shape[2] > 3:  # MS image, assuming more than 3 channels
            # averaging the bands
            image_gray = np.mean(image_array, axis=2)
        else:
            raise ValueError("The array does not have a recognized channel size for RGB or MS images.")
        
        # Extract HOG features
        features = hog(image_gray, orientations=8, pixels_per_cell=(16,16),
                       cells_per_block=(1,1), block_norm='L2-Hys')
    else:
        raise ValueError("The array is not three-dimensional as expected for an image.")
    
    return features

predictions = []
for test_id in test_df["test_id"]:
    image_npy_path = os.path.join(test_npy_path, f"test_{test_id}.npy")
    image_array = np.load(image_npy_path)
    
    features = preprocess_and_extract_features(image_array)
    
    predicted_label_index = clf.predict([features])[0]
    predicted_label = le.inverse_transform([predicted_label_index])[0]
    
    predictions.append(predicted_label)
    
submission_df = pd.DataFrame({
    "test_id": test_df["test_id"],
    "label": predictions
})

submission_csv_path = "./submission.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission file has been saved to {submission_csv_path}")



Submission file has been saved to ./submission.csv
